<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/BERT_binaryclassification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

source:https://www.youtube.com/watch?v=f-86-HcYYi8

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

from transformers import BertTokenizer, BertModel

In [25]:
df = pd.read_csv("./job_PII_diagnosis.csv", error_bad_lines=False)
df.head(5)

<ipython-input-25-dfcbf89df6b3>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("./job_PII_diagnosis.csv", error_bad_lines=False)


,prompt,related_to_job
0,My job is Marketing Intern and it's Marketing....,1
1,My job is Customer Service - Cloud Video Produ...,1
2,My job is Commissioning Machinery Assistant (C...,1
3,My job is Account Executive - Washington DC an...,1
4,My job is Bill Review Manager and it's freelan...,1


In [55]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 2
LEARNING_RATE = 1e-04

In [27]:
# tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [28]:
example = """
            My job is Marketing Intern and it's Marketing. It locates at US, NY, New York and the salary range is negotiable Following is the introduction of my company: We're Food52, and we've created a groundbreaking and award-winning cooking site. We support, connect, and celebrate home cooks, and give them everything they need in one place.We have a top editorial,, the job description: Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team o, the requirements: Experience with content management systems a major plus (any blogging counts!)Familiar with the Food52 editorial voice and aestheticLoves food, appreciates the importance of home cooking and cooking w, and the benefits: negotiable
        """

encodings = tokenizer.encode_plus(
    example,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensor='pt'
)
encodings

Keyword arguments {'return_tensor': 'pt'} not recognized.


{'input_ids': [101, 2026, 3105, 2003, 5821, 25204, 1998, 2009, 1005, 1055, 5821, 1012, 2009, 12453, 2015, 2012, 2149, 1010, 6396, 1010, 2047, 2259, 1998, 1996, 10300, 2846, 2003, 11265, 3995, 10711, 3468, 2206, 2003, 1996, 4955, 1997, 2026, 2194, 1024, 2057, 1005, 2128, 2833, 25746, 1010, 1998, 2057, 1005, 2310, 2580, 1037, 23222, 1998, 2400, 1011, 3045, 8434, 2609, 1012, 2057, 2490, 1010, 7532, 1010, 1998, 8439, 2188, 26929, 1010, 1998, 2507, 2068, 2673, 2027, 2342, 1999, 2028, 2173, 1012, 2057, 2031, 1037, 2327, 8368, 1010, 1010, 1996, 3105, 6412, 1024, 2833, 25746, 1010, 1037, 3435, 1011, 3652, 1010, 2508, 10154, 2400, 1011, 3045, 3784, 2833, 2451, 1998, 4306, 1011, 23184, 1998, 17940, 17974, 9594, 1010, 2003, 2747, 27805, 2440, 1011, 1998, 2112, 1011, 2051, 23850, 25204, 2015, 2000, 2147, 1999, 1037, 2235, 2136, 1051, 1010, 1996, 5918, 1024, 3325, 2007, 4180, 2968, 3001, 1037, 2350, 4606, 1006, 2151, 9927, 4726, 9294, 999, 1007, 5220, 2007, 1996, 2833, 25746, 8368, 2376, 1998, 1246

In [29]:
# df['prompt'][0]
# " ".join(str(df['prompt'][0]).split())

In [30]:
df['related_to_job'].values

array([1, 1, 1, ..., 0, 0, 0])

In [31]:
# encapsulte data with pytorch dataset
# help(tokenizer.encode_plus)

class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.max_len = max_len
        self.example = df['prompt'] #input X
        self.targets = df['related_to_job'].values #target y

    def __len__(self):
        return len(self.example)

    def __getitem__(self, index):
        example = str(self.example[index]) #in case any type error
        example = " ".join(example.split())

        inputs = self.tokenizer.encode_plus(
            example,
            None,
            add_special_tokens=True, #[CLS], [SEP], [PAD]
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'targets': torch.tensor([self.targets[index]], dtype=torch.float)
        }

In [32]:
# take only training and validation
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=200)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [33]:
print(val_df.describe())
print(val_df.isnull().sum())
print(train_df.describe())
print(train_df.isnull().sum())

       related_to_job
count     1934.000000
mean         0.631851
std          0.482427
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
prompt            0
related_to_job    0
dtype: int64
       related_to_job
count     7733.000000
mean         0.617871
std          0.485939
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
prompt            0
related_to_job    0
dtype: int64


In [34]:
val_df[0:5]

,prompt,related_to_job
0,My job is Director of Engineering and it's fre...,1
1,My job is Office Administrator and it's freela...,1
2,My job is Senior Front End Engineer and it's E...,1
3,My job is Field Engineer and it's freelancing....,1
4,My job is Designer: Internship and it's freela...,1


In [35]:
# encapsulate pandas dataframe with tokenizer and dataset
train_dataset = CustomDataSet(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataSet(val_df, tokenizer, MAX_LEN)

In [36]:
valid_dataset[1]['targets']

tensor([1.])

In [37]:
#dataset -> each row
def print_sample_shapes(dataset, num_samples=5):
    for i in range(num_samples):
        sample = dataset[i]
        print(f"Sample {i}:")
        print(f"  input_ids shape: {sample['input_ids'].shape}")
        print(f"  token_type_ids shape: {sample['token_type_ids'].shape}")
        print(f"  attention_mask shape: {sample['attention_mask'].shape}")
        print(f"  targets shape: {sample['targets'].shape}")
        print(sample['targets'])
        print("---------------")

# Print shapes for a few samples from the training dataset
print("Training Dataset Sample Shapes:")
print_sample_shapes(train_dataset)

# Print shapes for a few samples from the validation dataset
print("Validation Dataset Sample Shapes:")
print_sample_shapes(valid_dataset)

Training Dataset Sample Shapes:
Sample 0:
  input_ids shape: torch.Size([256])
  token_type_ids shape: torch.Size([256])
  attention_mask shape: torch.Size([256])
  targets shape: torch.Size([1])
tensor([1.])
---------------
Sample 1:
  input_ids shape: torch.Size([256])
  token_type_ids shape: torch.Size([256])
  attention_mask shape: torch.Size([256])
  targets shape: torch.Size([1])
tensor([0.])
---------------
Sample 2:
  input_ids shape: torch.Size([256])
  token_type_ids shape: torch.Size([256])
  attention_mask shape: torch.Size([256])
  targets shape: torch.Size([1])
tensor([0.])
---------------
Sample 3:
  input_ids shape: torch.Size([256])
  token_type_ids shape: torch.Size([256])
  attention_mask shape: torch.Size([256])
  targets shape: torch.Size([1])
tensor([1.])
---------------
Sample 4:
  input_ids shape: torch.Size([256])
  token_type_ids shape: torch.Size([256])
  attention_mask shape: torch.Size([256])
  targets shape: torch.Size([1])
tensor([1.])
---------------
Val

In [38]:
# Dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

In [39]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [40]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [47]:
# building model
class BERTclass(nn.Module):
    def __init__(self):
        super(BERTclass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, 1)

    def forward(self, input_ids, attn_mask, token_type_ids):
        x = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        x = self.dropout(x[0][:, 0, :]) #CLS token
        x = self.linear(x)
        return x

model = BERTclass()
model.to(device)

BERTclass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [48]:
#loss function
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets) # pass (outputs, targets) as argument and combining sigmoid function.
# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [49]:
val_targets=[]
val_outputs=[]

In [53]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    for epochs in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epochs))
        for index, batch in enumerate(training_loader):
            input_ids = batch['input_ids'].to(device, dtype = torch.long) #to gpu and take integer
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.float)
            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (index + 1)) * (loss.item() - train_loss))

        print('############# Epoch {}: Training End     #############'.format(epochs))
        print('############# Epoch {}: Validation Start   #############'.format(epochs))

        model.eval()
        with torch.no_grad():
            for index, batch in enumerate(validation_loader):
                input_ids = batch['input_ids'].to(device, dtype = torch.long) #to gpu and take integer
                attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
                token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
                targets = batch['targets'].to(device, dtype = torch.float)
                outputs = model(input_ids, attention_mask, token_type_ids)
                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss  + ((1 / (index + 1)) * (loss.item() - valid_loss))

            print('############# Epoch {}: Validation End     #############'.format(epochs))
            # calculate average losses
            train_loss = train_loss/len(training_loader)
            valid_loss = valid_loss/len(validation_loader)
            # print training/validation statistics
            print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
                    epochs,
                    train_loss,
                    valid_loss
                    ))

            checkpoint = {
                    'epoch': epochs + 1,
                    'valid_loss_min': valid_loss,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()
            }

            save_ckp(checkpoint, False, checkpoint_path, best_model_path)

            ## TODO: save the model if validation loss has decreased
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
                # save checkpoint as best model
                save_ckp(checkpoint, True, checkpoint_path, best_model_path)
                valid_loss_min = valid_loss
        print('############# Epoch {}  Done   #############\n'.format(epochs))
    return model


In [51]:
ckpt_path = "/content/curr_ckpt"
best_model_path = "/content/best_model.pt"

In [54]:
trained_model = train_model(EPOCHS, train_dataloader, val_dataloader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000000 	Average Validation Loss: 0.000000
Validation loss decreased (inf --> 0.000000).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000000 	Average Validation Loss: 0.000000
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
############# Epoch 2  Done   #############



In [ ]:
#save the model
torch.save(trained_model.state_dict(), '/content/model_state_dict.pth')

In [ ]:
# Recreate the model structure
model = BERTclass()  # Make sure this is the same structure as the saved model

# Load the state dictionary
model.load_state_dict(torch.load('/content/model_state_dict.pth'))

# If you're loading the model for inference only
model.eval()